In [45]:
import spacy
import pandas as pd
import sklearn
import numpy as np
from scipy.spatial.distance import cosine
import random
import warnings
from __future__ import unicode_literals
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [46]:
en_nlp = spacy.load('en')

In [47]:
def get_vec(sentence):
    vec = np.zeros(384)
    for word in sentence:
        if word in wordvecs:
            vec += wordvecs[word]
    return vec
    
    

In [48]:
def spacy_get_vec(sentence):
    vec = np.zeros(384)
    doc = en_nlp((sentence))
    for word in doc:
        vec += word.vector 
    return vec

In [49]:
lines = open('./intents.txt').readlines()

In [50]:
vecs = []
intents = []
for line in lines:
    tokens = line.split(',')
    sentence = tokens[0]
    intent = tokens[1]
    if intent[-1] == '\n':
        intent = intent[:-1]
    vecs.append(spacy_get_vec(sentence))
    intents.append(intent)

In [51]:
df = pd.DataFrame(vecs, columns=['vec_%d' % i for i in range(384)])

In [52]:
df['intents'] = intents

In [53]:
df.intents = df.intents.astype('category')

In [54]:
X = df.iloc[:, :-1].values

In [55]:
y = df.iloc[:,-1:].values.ravel()

In [56]:
from sklearn.cross_validation import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X, y, test_size=0.20)

In [57]:
from sklearn.linear_model import LogisticRegression
logit_model = LogisticRegression(C=1.0)
logit_model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [58]:
logit_model.score(X_val, y_val)

0.58333333333333337

In [59]:
from sklearn.ensemble import GradientBoostingClassifier

In [60]:
gradboost = GradientBoostingClassifier(n_estimators=500, max_depth=25, max_features='sqrt')

In [61]:
gradboost.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=25,
              max_features=u'sqrt', max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=500, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [62]:
gradboost.score(X_val, y_val)

0.66666666666666663

In [70]:
sent = 'hi'
print(gradboost.predict(spacy_get_vec(sent)))
print(gradboost.predict_proba(spacy_get_vec(sent)))

[u'greeting']
[[  1.09722455e-04   8.79408295e-05   7.37595480e-05   9.99518853e-01
    1.01257941e-04   1.08466392e-04]]


In [71]:
sent = 'can you tell the forecast for tomorrow'
print(gradboost.predict(spacy_get_vec(sent)))
print(gradboost.predict_proba(spacy_get_vec(sent)))

[u'weather']
[[ 0.03445437  0.01284532  0.00806584  0.03408671  0.11150728  0.79904048]]


In [72]:
gradboost.classes_

array([u'bad', u'goodbye', u'greet_response', u'greeting', u'travel',
       u'weather'], dtype=object)

In [73]:
logit_model.predict_proba(spacy_get_vec('is there a hailstorm outside').reshape(1,-1))

array([[  1.66265950e-03,   2.40823278e-04,   6.83378409e-06,
          1.88385986e-03,   1.04709722e-06,   9.96204776e-01]])

In [77]:
gradboost.fit(X,y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=25,
              max_features=u'sqrt', max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=500, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [78]:
from sklearn.externals import joblib
joblib.dump(gradboost, 'intent.pkl')

[u'intent.pkl']

In [75]:
logit_model.predict(spacy_get_vec('Infinity'))

array([u'greeting'], dtype=object)

In [76]:
gradboost.predict(spacy_get_vec('Infinity'))

array([u'greeting'], dtype=object)